In [92]:
import pandas as pd
import numpy as np
import googlemaps
import os

### 데이터 불러오기

In [93]:
BNC = pd.read_excel("./data/NSC2_BNC_1619.xlsx", usecols=[ "RN_INDI", "STD_YYYY", "SGG"])            # RN_INDI, STD_YYYY, SGG
INST = pd.read_excel("./data/NSC2_INST_1619.xlsx", usecols=["STD_YYYY", "RN_INST", "SGG_INST"])     # STD_YYYY, RN_INST, SGG_INST

C:\Users\Byeo\anaconda3\envs\Lec_DataMining\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [94]:
T20_dir = "./data/20T"
T20_list= os.listdir(T20_dir)
T20 = pd.DataFrame(columns=["RN_INDI", "RN_INST", "MCARE_SUBJ_CD", "STD_YYYY"])
for name in T20_list:
    print(name + "불러오기 시작")
    temp = pd.read_excel(T20_dir + "/" + name, usecols=["RN_INDI", "RN_INST", "MCARE_SUBJ_CD", "STD_YYYY"])  # RN_INDI, RN_INST,MCARE_SUBJ_CD, STD_YYYY
    temp.drop(index=0, axis=0)
    T20 = pd.concat([T20, temp])
    print("완료")
T20 = T20[["STD_YYYY", "RN_INDI", "RN_INST", "MCARE_SUBJ_CD"]]

NSC2_D20_1000.xlsx불러오기 시작
완료
NSC2_K20_1000.xlsx불러오기 시작
완료
NSC2_M20_1000.xlsx불러오기 시작
완료
NSC2_P20_1000.xlsx불러오기 시작
완료


### 중복제거 및 2015년 필터링

In [95]:
# BNC = BNC.drop_duplicates("RN_INDI")
INST = INST.drop_duplicates("RN_INST").sort_values(by="STD_YYYY", axis=0, ascending=False).reset_index(drop=True)
BNC = BNC[["STD_YYYY", "RN_INDI", "SGG"]]

BNC2019 = BNC[BNC["STD_YYYY"] == "_x0032_015"].sort_values(by=['RN_INDI'], axis=0).reset_index(drop=True)
INST2019 = INST[INST["STD_YYYY"] == "_x0032_015"].sort_values(by=['RN_INST'], axis=0).reset_index(drop=True)
T202019 = T20[T20["STD_YYYY"] == "_x0032_015"].sort_values(by=['RN_INDI'], axis=0).reset_index(drop=True)

In [96]:
print("BNC")
print(BNC2019.head())

print("\nINST")
print(INST2019.head())

print("\nT20")
print(T202019)

BNC
     STD_YYYY  RN_INDI          SGG
0  _x0032_015     2270  _x0034_5111
1  _x0032_015     3317  _x0034_1610
2  _x0032_015     3690  _x0034_1113
3  _x0032_015     4263  _x0034_6790
4  _x0032_015     4664  _x0031_1215

INST
     STD_YYYY  RN_INST     SGG_INST
0  _x0032_015       14  _x0034_1131
1  _x0032_015       45  _x0034_1410
2  _x0032_015       70  _x0033_6110
3  _x0032_015       95  _x0033_6110
4  _x0032_015      155  _x0031_1560

T20
         STD_YYYY  RN_INDI RN_INST MCARE_SUBJ_CD
0      _x0032_015     2270   72384      _x0035_4
1      _x0032_015     3317   65702      _x0031_4
2      _x0032_015     3317   44113      _x0030_0
3      _x0032_015     3317  114238      _x0031_2
4      _x0032_015     3317   44113      _x0030_0
...           ...      ...     ...           ...
25700  _x0032_015  3001120   78129      _x0030_1
25701  _x0032_015  3001120   37310      _x0030_1
25702  _x0032_015  3001120   44764      _x0035_5
25703  _x0032_015  3001120   78129      _x0030_1
25704  _x0032_

### 결과데이터에 개인주소지 및 기관주소지 추가 + 이상치( 정보데이터가 없거나 주소지와 방문기관주소지가 같은 관측치 ) 제거

In [97]:
inputData = T202015.copy()
compare_BNC = BNC2015.copy()
compare_INST = INST2015.copy()
compare_INST = INST.copy()

result_SSG = []
result_SSG_INST = []
print(str(len(inputData)) + " 개의 데이터가 있습니다.")
print("처리중", end='')

for idx in range(len(inputData)):
    if(idx%1000==0):
        print(".", end='')
    localValue_INDI = inputData.iloc[idx]["RN_INDI"]
    localValue_INST = inputData.iloc[idx]["RN_INST"]
    SGG_idx = compare_BNC.index[(compare_BNC["RN_INDI"] == localValue_INDI)][0]

    # 값 없는 부분 예외처리
    try:
        SGG_INST_idx = compare_INST.index[(compare_INST["RN_INST"] == localValue_INST)][0]
    except:
        SGG_INST_idx = -1

    # 결과 데이터 저장( Append )
    result_SSG.append(compare_BNC.iloc[SGG_idx]["SGG"])

    if(SGG_INST_idx != -1):
        result_SSG_INST.append(compare_INST.iloc[SGG_INST_idx]["SGG_INST"])
    else:
        result_SSG_INST.append("-1")

result_data = inputData.copy()
result_data["SSG"] = result_SSG
result_data["SSG_INST"] = result_SSG_INST

delete_idx = result_data.index[(result_data["SSG_INST"] == "-1")]
result_data = result_data.drop(delete_idx)

delete_idx = result_data.index[(result_data["SSG"] == result_data["SSG_INST"])]
result_data = result_data.drop(delete_idx).reset_index(drop=True)

print("완료")


25705 개의 데이터가 있습니다.
처리중..........................완료


### 파일저장

In [98]:
# result_data.to_json("test.json", orient='table')

### 위도 경도 데이터 구성


In [99]:
import math

In [146]:
data_base = result_data.copy()
location_table = pd.read_excel("./data/SSG_processed.xlsx").astype({"SSG": str})

print(data_base.head())
print(location_table.head())

     STD_YYYY RN_INDI RN_INST MCARE_SUBJ_CD          SSG     SSG_INST
0  _x0032_015    3690   19717      _x0030_0  _x0034_1113  _x0034_4133
1  _x0032_015    3690   18640      _x0030_0  _x0034_1113  _x0034_1590
2  _x0032_015    3690  115628      _x0030_5  _x0034_1113  _x0034_1590
3  _x0032_015    3690   34091      _x0038_0  _x0034_1113  _x0034_1115
4  _x0032_015    3690  112956      _x0038_0  _x0034_1113  _x0034_4810
     SSG     시도  시군구        lat         lng
0  11110  서울특별시  종로구  37.572950  126.979358
1  11140  서울특별시   중구  37.564091  126.997940
2  11170  서울특별시  용산구  37.538427  126.965444
3  11200  서울특별시  성동구  37.563341  127.037103
4  11215  서울특별시  광진구  37.538484  127.082294


In [147]:
distance = []

print(str(len(data_base)) + " 개의 데이터가 있습니다.")
print("처리중", end='')
for idx in range(len(data_base)):
    if(idx%500 == 0):
        print(".", end='')
    INDI_SSG = data_base.iloc[idx]["SSG"]
    INDI_SSG = INDI_SSG[5:].replace("_","")
    INDI_SGG_idx = location_table.index[(location_table["SSG"] == INDI_SSG)][0]
    INDI_lat = location_table.iloc[INDI_SGG_idx]["lat"]
    INDI_lng = location_table.iloc[INDI_SGG_idx]["lng"]

    INST_SSG = data_base.iloc[idx]["SSG_INST"]
    INST_SSG = INST_SSG[5:].replace("_","")
    INST_SGG_idx = location_table.index[(location_table["SSG"] == INST_SSG)][0]
    INST_lat = location_table.iloc[INST_SGG_idx]["lat"]
    INST_lng = location_table.iloc[INST_SGG_idx]["lng"]

    cal_result = math.sqrt(((INDI_lat-INST_lat)*111)**2 + ((INDI_lng-INST_lng)*88.8)**2)
    # print( "{( " + str(INDI_lat) + " - " + str(INST_lat) + " ) x 111}^2 {(" + str(INDI_lng) + " - " + str(INST_lng) + ") x 88.8 }**2" )

    distance.append(round(cal_result,3))
print("끝!")
result = data_base.copy()
result["DISTANCE"] = distance

11418 개의 데이터가 있습니다.
처리중.......................끝!


In [148]:
print(result.head())
result.to_excel("test3_distance.xlsx")

     STD_YYYY RN_INDI RN_INST MCARE_SUBJ_CD          SSG     SSG_INST  \
0  _x0032_015    3690   19717      _x0030_0  _x0034_1113  _x0034_4133   
1  _x0032_015    3690   18640      _x0030_0  _x0034_1113  _x0034_1590   
2  _x0032_015    3690  115628      _x0030_5  _x0034_1113  _x0034_1590   
3  _x0032_015    3690   34091      _x0038_0  _x0034_1113  _x0034_1115   
4  _x0032_015    3690  112956      _x0038_0  _x0034_1113  _x0034_4810   

   DISTANCE  
0    44.684  
1    14.066  
2    14.066  
3     6.433  
4    64.721  
